In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm
import re
from sklearn.preprocessing import LabelEncoder
import pickle
import os

/Users/jquinn/repos/cs4262-report/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Analysis

In [2]:
rule_col = "rule"
body_col = "body"  
label_col = "rule_violation"

In [3]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")  
solution = pd.read_csv("data/solution.csv")  

In [4]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              2029 non-null   int64 
 1   body                2029 non-null   object
 2   rule                2029 non-null   object
 3   subreddit           2029 non-null   object
 4   positive_example_1  2029 non-null   object
 5   positive_example_2  2029 non-null   object
 6   negative_example_1  2029 non-null   object
 7   negative_example_2  2029 non-null   object
 8   rule_violation      2029 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 142.8+ KB


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [5]:
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54059 entries, 0 to 54058
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              54059 non-null  int64 
 1   body                54059 non-null  object
 2   rule                54059 non-null  object
 3   subreddit           54059 non-null  object
 4   positive_example_1  54059 non-null  object
 5   positive_example_2  54059 non-null  object
 6   negative_example_1  54059 non-null  object
 7   negative_example_2  54059 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.3+ MB


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,Hi Ken l! /u/Whimsical_wispers told me he want...,No promotion of illegal activity: Do not encou...,IAmA,There are several web sites on internet that a...,Someone needs to rape that judge and see how w...,call whores watch for you herein http://onj.me...,52 http://MySexFind.com - find girl for sex no...
1,2030,Doubt anyone will read this but anti depressan...,No medical advice: Do not offer or request spe...,science,Maybe he just has alzheimer's. It would explai...,Hmm well I WAS going to ask women of Reddit ho...,You could always just carry your kid and get t...,Maybe you should get started on the solution a...
2,2031,**[One of her best show](http://shortora.com/i...,"No Advertising: Spam, referral links, unsolici...",NSFW_GIF,Son Dakika Ek Kazanç Videolarımızı İzleyin :\n...,Try My New Game..! :)\nhttps://play.google.com...,**HD** Stream[English Tiwtch](http://www.zifoo...,/r/krat0m <-subscribe for free kilo of strong ...
3,2032,If he'd cheat on you he'll cheat on her. She's...,No spoilers: Do not reveal important details t...,relationships,And now Jessica is gone. They really need to ...,"Probably a meme at this point, particularly th...","hahah. Please, Barry changed his name to hide ...",I found the full video.\nThere were a few peop...
4,2033,I think I remember reading somewhere that tech...,No legal advice: Do not offer or request legal...,AskReddit,"I mean, even if they didn't have anything sign...","Fire her. You can't be sued for it, only the c...",Personally. I'd be wrong to tell you suicide i...,"I call it salary, perhaps a better term is sch..."


In [6]:
print("\nTraining Set")
print(f"Total samples: {len(train)}")
print(f"Class distribution:\n{train[label_col].value_counts()}")
print(f"Class ratio: {train[label_col].value_counts(normalize=True)}")
print(f"\nUnique rules in train: {train[rule_col].nunique()}")
print(f"Rules: {train[rule_col].unique()}")
print(f"\nUnique subreddits in train: {train['subreddit'].nunique()}")

print("\nTest Set")
print(f"Total samples: {len(test)}")
print(f"\nUnique rules in test: {test[rule_col].nunique()}")
print(f"Rules: {test[rule_col].unique()}")
print(f"\nUnique subreddits in test: {test['subreddit'].nunique()}")

print("\nAnalysis")
train_rules = set(train[rule_col].unique())
test_rules = set(test[rule_col].unique())
print(f"Rules in test but NOT in train: {test_rules - train_rules}")
print(f"Number of test samples with unseen rules: {test[~test[rule_col].isin(train_rules)].shape[0]}")
print(f"Percentage: {100 * test[~test[rule_col].isin(train_rules)].shape[0] / len(test):.2f}%")

train_subreddits = set(train['subreddit'].unique())
test_subreddits = set(test['subreddit'].unique())


Training Set
Total samples: 2029
Class distribution:
rule_violation
1    1031
0     998
Name: count, dtype: int64
Class ratio: rule_violation
1    0.508132
0    0.491868
Name: proportion, dtype: float64

Unique rules in train: 2
Rules: ['No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'
 'No legal advice: Do not offer or request legal advice.']

Unique subreddits in train: 100

Test Set
Total samples: 54059

Unique rules in test: 6
Rules: ['No promotion of illegal activity: Do not encourage or promote illegal activities, such as drug-related activity, violence, exploitation, theft, or other criminal behavior.'
 'No medical advice: Do not offer or request specific medical advice, diagnoses, or treatment recommendations.'
 'No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'
 "No spoilers: Do not reveal important details that would limit people's ability to enjoy a show or movie."

During our data analysis we noticed a few pain points in solving this problem. First, we have a small training dataset of 2,029 samples, and we are evaluating this on a test set of 54,059 samples. Secondly, we have an additional 4 rules in our test dataset that are unseen in our training dataset, these unseen rules make up 66.76% of our test samples

# Training Helper Functions

In [6]:
device = 'cuda' if os.system('nvidia-smi > /dev/null 2>&1') == 0 else 'mps' if os.system('sysctl -n machdep.cpu.brand_string | grep -q Apple > /dev/null 2>&1') == 0 else 'cpu'
print(f"Using device: {device}")

model = SentenceTransformer("all-mpnet-base-v2", device=device)

def embed_batch(texts):
    return model.encode(texts, normalize_embeddings=True, show_progress_bar=True)

Using device: mps


In [7]:
def load_or_create_embeddings(df, prefix, cache_dir="embeddings_cache"):
    os.makedirs(cache_dir, exist_ok=True)
    embeddings = {}

    to_embed = {
        'body': body_col,
        'pos1': 'positive_example_1',
        'pos2': 'positive_example_2',
        'neg1': 'negative_example_1',
        'neg2': 'negative_example_2'
    }

    for key, col in to_embed.items():
        cache_file = f"{cache_dir}/{prefix}_{key}_emb.pkl"

        if os.path.exists(cache_file):
            print(f"Loading cached {prefix} {key} embeddings...")
            embeddings[f"{key}_emb"] = pickle.load(open(cache_file, 'rb'))
        else:
            print(f"Embedding {prefix} {col}...")
            embeddings[f"{key}_emb"] = embed_batch(df[col].tolist())
            pickle.dump(embeddings[f"{key}_emb"], open(cache_file, 'wb'))

    return embeddings



In [8]:

def extract_text_features(text):
    num_chars = len(text)
    num_words = len(text.split())

    has_url = 1 if re.search(r'http[s]?://|www\.', text) else 0

    if num_chars > 0:
        caps_ratio = sum(1 for c in text if c.isupper()) / num_chars
    else:
        caps_ratio = 0

    return np.array([num_chars, num_words, has_url, caps_ratio])

In [9]:
def combine_features(body, rule, pos1, pos2, neg1, neg2, text, subreddit_encoded):
    sim_pos1 = (body * pos1).sum()
    sim_pos2 = (body * pos2).sum()
    sim_neg1 = (body * neg1).sum()
    sim_neg2 = (body * neg2).sum()
    sim_rule = (body * rule).sum()

    pos_avg = (sim_pos1 + sim_pos2) / 2
    neg_avg = (sim_neg1 + sim_neg2) / 2
    pos_neg_ratio = pos_avg / (neg_avg + 1e-6)
    pos_neg_diff = pos_avg - neg_avg

    text_feats = extract_text_features(text)

    return np.concatenate([
        [sim_pos1, sim_pos2, sim_neg1, sim_neg2, sim_rule],  # 5 features
        [pos_neg_ratio, pos_neg_diff],  # 2 features
        text_feats,  # 4 features (num_chars, num_words, has_url, caps_ratio)
        [subreddit_encoded]  # 1 feature
    ])

In [10]:
print("Embedding unique rules...")
all_unique_rules = pd.concat([train[rule_col], test[rule_col]]).unique()
unique_rule_emb = embed_batch(all_unique_rules.tolist())
rule_to_emb = dict(zip(all_unique_rules, unique_rule_emb))
rule_emb = np.array([rule_to_emb[rule] for rule in train[rule_col]])

print("\nEmbedding train set")
train_emb = load_or_create_embeddings(train, prefix='train')
body_emb = train_emb['body_emb']
pos1_emb = train_emb['pos1_emb']
pos2_emb = train_emb['pos2_emb']
neg1_emb = train_emb['neg1_emb']
neg2_emb = train_emb['neg2_emb']
rule_emb_test = np.array([rule_to_emb[rule] for rule in test[rule_col]])

print("\nEmbedding test set")
test_emb = load_or_create_embeddings(test, prefix='test')
body_emb_test = test_emb['body_emb']
pos1_emb_test = test_emb['pos1_emb']
pos2_emb_test = test_emb['pos2_emb']
neg1_emb_test = test_emb['neg1_emb']
neg2_emb_test = test_emb['neg2_emb']

Embedding unique rules...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.84it/s]



Embedding train set
Loading cached train body embeddings...
Loading cached train pos1 embeddings...
Loading cached train pos2 embeddings...
Loading cached train neg1 embeddings...
Loading cached train neg2 embeddings...

Embedding test set
Loading cached test body embeddings...
Loading cached test pos1 embeddings...
Loading cached test pos2 embeddings...
Loading cached test neg1 embeddings...
Loading cached test neg2 embeddings...


In [11]:
subreddit_encoder = LabelEncoder()
subreddit_encoder.fit(pd.concat([train['subreddit'], test['subreddit']]))

LabelEncoder()

In [12]:
train_subreddit_encoded = subreddit_encoder.transform(train['subreddit'])
y = train[label_col].values

X = np.array([
    combine_features(
        body_emb[i], rule_emb[i], 
        pos1_emb[i], pos2_emb[i], 
        neg1_emb[i], neg2_emb[i],
        train[body_col].iloc[i],
        train_subreddit_encoded[i]
    ) 
    for i in range(len(train))
])


print(f"Feature shape: {X.shape}")
X.shape, y.shape

Feature shape: (2029, 12)


((2029, 12), (2029,))

In [13]:
test_subreddit_encoded = subreddit_encoder.transform(test['subreddit'])

X_test = np.array([
    combine_features(
        body_emb_test[i], rule_emb_test[i],
        pos1_emb_test[i], pos2_emb_test[i],
        neg1_emb_test[i], neg2_emb_test[i],
        test[body_col].iloc[i],
        test_subreddit_encoded[i]
    )
    for i in range(len(test))
])

print(f"Test features shape: {X_test.shape}")

Test features shape: (54059, 12)


# Main Training Loop

In [15]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, n_jobs=-1, class_weight="balanced"),
    'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=10, n_jobs=-1, class_weight="balanced", random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, reg_alpha=1.0, reg_lambda=1.0, random_state=42, n_jobs=-1, eval_metric='logloss'),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
}

results = {}

for name, model in models.items():
    print(f"Training {name}...")

    model.fit(X, y)
    
    test_preds = model.predict(X_test)
    test_probs = model.predict_proba(X_test)[:,1]
    
    test_results = test[["row_id"]].copy()
    test_results["prediction"] = test_probs
    test_results = test_results.merge(solution[["row_id", "rule_violation"]], on="row_id", how="left")
    y_test = test_results["rule_violation"].values
    
    print(f"Metrics on test dataset")
    print(classification_report(y_test, test_preds))
    test_auc = roc_auc_score(y_test, test_probs)
    test_acc = (test_preds == y_test).mean()
    print(f"AUC: {test_auc:.4f}")
    print(f"Accuracy: {test_acc:.4f}")
    
    results[name] = {
        'test_auc': test_auc,
        'test_acc': test_acc,
        'model': model,
        'test_probs': test_probs
    }
    print("\n")

print("Summary")
for name, res in results.items():
    print(f"{name:20s} | Test AUC: {res['test_auc']:.4f} | Test Acc: {res['test_acc']:.4f}")

# best_model_name = max(results.items(), key=lambda x: x[1]['test_auc'])[0]
# best_probs = results[best_model_name]['test_probs']
# test[["row_id"]].assign(prediction=best_probs).to_csv("submission.csv", index=False)
# print(f"\nSaved submission.csv using {best_model_name}")

Training Logistic Regression...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.68      0.68      0.68     28789
           1       0.64      0.63      0.63     25270

    accuracy                           0.66     54059
   macro avg       0.66      0.66      0.66     54059
weighted avg       0.66      0.66      0.66     54059

AUC: 0.7194
Accuracy: 0.6589


Training Random Forest...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.68      0.64      0.66     28789
           1       0.61      0.66      0.64     25270

    accuracy                           0.65     54059
   macro avg       0.65      0.65      0.65     54059
weighted avg       0.65      0.65      0.65     54059

AUC: 0.7079
Accuracy: 0.6476


Training Gradient Boosting...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.64      0.63      0.64     28789
           1   

## 2. More sophisticated similarity features

In [21]:
def combine_features_v2(body, rule, pos1, pos2, neg1, neg2, text, subreddit_encoded):
    # Basic similarities 
    sim_pos1 = (body * pos1).sum()
    sim_pos2 = (body * pos2).sum()
    sim_neg1 = (body * neg1).sum()
    sim_neg2 = (body * neg2).sum()
    sim_rule = (body * rule).sum()
    
    # Aggregate similarities
    pos_avg = (sim_pos1 + sim_pos2) / 2
    neg_avg = (sim_neg1 + sim_neg2) / 2
    pos_max = max(sim_pos1, sim_pos2)
    pos_min = min(sim_pos1, sim_pos2)
    neg_max = max(sim_neg1, sim_neg2)
    neg_min = min(sim_neg1, sim_neg2)
    
    # Ratio and difference features
    pos_neg_ratio = pos_avg / (neg_avg + 1e-6)
    pos_neg_diff = pos_avg - neg_avg
    pos_range = pos_max - pos_min  # Consistency of positive examples
    neg_range = neg_max - neg_min  # Consistency of negative examples
    
    # New: Compare with rule embedding
    rule_pos_avg_sim = (rule * ((pos1 + pos2) / 2)).sum()
    rule_neg_avg_sim = (rule * ((neg1 + neg2) / 2)).sum()
    
    # New: Cross-similarities between examples
    pos1_pos2_sim = (pos1 * pos2).sum()
    neg1_neg2_sim = (neg1 * neg2).sum()
    
    # New: Distance-based features (L2 distance)
    pos_avg_emb = (pos1 + pos2) / 2
    neg_avg_emb = (neg1 + neg2) / 2
    dist_to_pos = np.linalg.norm(body - pos_avg_emb)
    dist_to_neg = np.linalg.norm(body - neg_avg_emb)
    dist_ratio = dist_to_neg / (dist_to_pos + 1e-6)
    
    # Text statistical features
    text_feats = extract_text_features(text)
    
    # Combine all features
    return np.concatenate([
        # Original similarities (5)
        [sim_pos1, sim_pos2, sim_neg1, sim_neg2, sim_rule],
        # Aggregate features (6)
        [pos_avg, neg_avg, pos_max, neg_max, pos_min, neg_min],
        # Ratio/diff features (4)
        [pos_neg_ratio, pos_neg_diff, pos_range, neg_range],
        # Rule-example similarities (2)
        [rule_pos_avg_sim, rule_neg_avg_sim],
        # Cross-similarities (2)
        [pos1_pos2_sim, neg1_neg2_sim],
        # Distance features (3)
        [dist_to_pos, dist_to_neg, dist_ratio],
        # Text features (4)
        text_feats,
        # Categorical (1)
        [subreddit_encoded]
    ])

# Build improved features for train
print("Building enhanced training features...")
X_v2 = np.array([
    combine_features_v2(
        body_emb[i], rule_emb[i], 
        pos1_emb[i], pos2_emb[i], 
        neg1_emb[i], neg2_emb[i],
        train[body_col].iloc[i],
        train_subreddit_encoded[i]
    ) 
    for i in range(len(train))
])

print(f"Enhanced feature shape: {X_v2.shape}")
print(f"Feature breakdown: 5 basic + 6 aggregate + 4 ratio/diff + 2 rule-example + 2 cross + 3 distance + 4 text + 1 categorical = {X_v2.shape[1]} total")

# Build improved features for test
print("\nBuilding enhanced test features...")
X_test_v2 = np.array([
    combine_features_v2(
        body_emb_test[i], rule_emb_test[i],
        pos1_emb_test[i], pos2_emb_test[i],
        neg1_emb_test[i], neg2_emb_test[i],
        test[body_col].iloc[i],
        test_subreddit_encoded[i]
    )
    for i in range(len(test))
])

print(f"Enhanced test feature shape: {X_test_v2.shape}")


Building enhanced training features...
Enhanced feature shape: (2029, 27)
Feature breakdown: 5 basic + 6 aggregate + 4 ratio/diff + 2 rule-example + 2 cross + 3 distance + 4 text + 1 categorical = 27 total

Building enhanced test features...
Enhanced test feature shape: (54059, 27)


In [22]:
models = {
    'Logistic Regression': LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        class_weight="balanced"
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        n_jobs=-1,
        class_weight="balanced",
        random_state=42
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        random_state=42
    ),
    'XGBoost': XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1.0,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    ),
    'SVM (RBF)': SVC(
        kernel='rbf',
        probability=True,
        class_weight='balanced',
        random_state=42
    )
}

results = {}

# Pre-merge test labels once
y_test = (
    test[['row_id']]
    .merge(solution[['row_id', 'rule_violation']], on='row_id', how='left')
    ['rule_violation']
    .values
)

for name, model in models.items():
    print(f"Training {name}...")

    # Train on the new feature set
    model.fit(X_v2, y)

    # Predict
    test_preds = model.predict(X_test_v2)
    test_probs = model.predict_proba(X_test_v2)[:, 1]

    # Evaluation
    print("Metrics on test dataset")
    print(classification_report(y_test, test_preds))

    test_auc = roc_auc_score(y_test, test_probs)
    test_acc = (test_preds == y_test).mean()

    print(f"AUC: {test_auc:.4f}")
    print(f"Accuracy: {test_acc:.4f}")
    print()

    results[name] = {
        'test_auc': test_auc,
        'test_acc': test_acc,
        'model': model,
        'test_probs': test_probs
    }

print("Summary")
for name, res in results.items():
    print(f"{name:20s} | Test AUC: {res['test_auc']:.4f} | Test Acc: {res['test_acc']:.4f}")

# Example save block (unchanged)
# best_model_name = max(results.items(), key=lambda x: x[1]['test_auc'])[0]
# best_probs = results[best_model_name]['test_probs']
# test[['row_id']].assign(prediction=best_probs).to_csv("submission.csv", index=False)
# print(f"\nSaved submission.csv using {best_model_name}")


Training Logistic Regression...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.67      0.70      0.68     28789
           1       0.64      0.61      0.62     25270

    accuracy                           0.66     54059
   macro avg       0.66      0.65      0.65     54059
weighted avg       0.66      0.66      0.66     54059

AUC: 0.7159
Accuracy: 0.6568

Training Random Forest...


/Users/jquinn/repos/cs4262-report/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Metrics on test dataset
              precision    recall  f1-score   support

           0       0.70      0.64      0.67     28789
           1       0.63      0.69      0.66     25270

    accuracy                           0.67     54059
   macro avg       0.67      0.67      0.67     54059
weighted avg       0.67      0.67      0.67     54059

AUC: 0.7245
Accuracy: 0.6652

Training Gradient Boosting...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.64      0.64      0.64     28789
           1       0.59      0.60      0.59     25270

    accuracy                           0.62     54059
   macro avg       0.62      0.62      0.62     54059
weighted avg       0.62      0.62      0.62     54059

AUC: 0.6689
Accuracy: 0.6197

Training XGBoost...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.67      0.65      0.66     28789
           1       0.61      0.63      0.62     25270



## 3. Ensemble of Multiple Models

In [23]:
print("Ensamble approach")

# Create ensemble of diverse models
ensemble_models = [
    ('logistic', LogisticRegression(C=clf_cv.C_[0], max_iter=1000, class_weight='balanced', random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=8, min_samples_leaf=10, 
                                   class_weight='balanced', random_state=42, n_jobs=-1)),
    ('gb', GradientBoostingClassifier(n_estimators=150, max_depth=4, learning_rate=0.05, 
                                       subsample=0.8, random_state=42))
]


ensemble = VotingClassifier(
    estimators=ensemble_models,
    voting='soft'
)

print(f"Training ensemble of {len(ensemble_models)} models...")
ensemble.fit(X_v2, y)

test_probs_ensemble = ensemble.predict_proba(X_test_v2)[:,1]
test_preds_ensemble = ensemble.predict(X_test_v2)

print(f"\nEnsemble Test Performance:")
print(classification_report(y_test, test_preds_ensemble))
test_auc_ensemble = roc_auc_score(y_test, test_probs_ensemble)
print(f"Test AUC: {test_auc_ensemble:.4f}")

Ensamble approach


NameError: name 'VotingClassifier' is not defined

In [ ]:
"""
ADDITIONAL IMPROVEMENT IDEAS TO TRY:

1. BETTER EMBEDDING MODELS:
   - all-MiniLM-L12-v2: Faster, slightly worse quality
   - paraphrase-mpnet-base-v2: Specifically trained for semantic similarity
   - all-distilroberta-v1: Good balance of speed/quality
   - sentence-t5-large: Larger model, better quality (slower)
   
2. RULE-SPECIFIC APPROACHES:
   Since test has unseen rules, consider:
   - Train separate models per rule (if enough samples)
   - Use rule embeddings as features more heavily
   - Compute similarity between new rules and training rules
   
3. DATA AUGMENTATION:
   - Paraphrase training examples using LLMs
   - Back-translation augmentation
   - Mix positive/negative examples with noise
   
4. ITERATIVE PSEUDO-LABELING:
   - Start with high confidence (0.95+)
   - Retrain and gradually lower threshold
   - Multiple iterations
   
5. STACKING (Meta-Learning):
   - Train multiple base models
   - Use their predictions as features for a meta-model
   - Can capture complex model interactions
   
6. CONTRASTIVE LEARNING:
   - Fine-tune sentence embeddings on this specific task
   - Use positive/negative examples as training signal
   - Create triplet loss: (body, pos_example, neg_example)
   
7. CALIBRATION:
   - Use Platt scaling or isotonic regression
   - Calibrate probabilities to better match test distribution
   
8. FEATURE SELECTION:
   - Use L1 regularization to identify most important features
   - Remove noisy features that may hurt generalization
   
9. ANALYZE ERRORS:
   - Look at misclassified examples
   - Identify patterns in what the model gets wrong
   - Engineer features specifically for those cases

To implement any of these, uncomment and modify the code in the next cells!
"""

In [ ]:
# ============================================================================
# FEATURE SELECTION USING L1 REGULARIZATION
# ============================================================================

print("="*80)
print("FEATURE SELECTION WITH L1 REGULARIZATION")
print("="*80)

# First, check the actual feature count
print(f"\nActual feature matrix shape: {X_v2.shape}")
print(f"Number of features: {X_v2.shape[1]}")

# Create feature names matching the actual combine_features_v2 function
# Based on the concatenation order in combine_features_v2
feature_names = []

# Original similarities (5)
feature_names.extend(['sim_pos1', 'sim_pos2', 'sim_neg1', 'sim_neg2', 'sim_rule'])

# Aggregate features (6)
feature_names.extend(['pos_avg', 'neg_avg', 'pos_max', 'neg_max', 'pos_min', 'neg_min'])

# Ratio/diff features (4)
feature_names.extend(['pos_neg_ratio', 'pos_neg_diff', 'pos_range', 'neg_range'])

# Rule-example similarities (2)
feature_names.extend(['rule_pos_avg_sim', 'rule_neg_avg_sim'])

# Cross-similarities (2)
feature_names.extend(['pos1_pos2_sim', 'neg1_neg2_sim'])

# Distance features (3)
feature_names.extend(['dist_to_pos', 'dist_to_neg', 'dist_ratio'])

# Text features (4)
feature_names.extend(['num_chars', 'num_words', 'has_url', 'caps_ratio'])

# Categorical (1)
feature_names.extend(['subreddit_encoded'])

# If there's still a mismatch, add placeholder names
if len(feature_names) < X_v2.shape[1]:
    print(f"\nWARNING: Expected {len(feature_names)} features but found {X_v2.shape[1]}")
    print(f"Adding {X_v2.shape[1] - len(feature_names)} placeholder feature names...")
    for i in range(len(feature_names), X_v2.shape[1]):
        feature_names.append(f'feature_{i}')
elif len(feature_names) > X_v2.shape[1]:
    print(f"\nWARNING: Have {len(feature_names)} feature names but only {X_v2.shape[1]} features")
    print(f"Truncating to {X_v2.shape[1]} names...")
    feature_names = feature_names[:X_v2.shape[1]]

print(f"Feature names created: {len(feature_names)}")

# Train L1-regularized Logistic Regression
print("\n" + "="*60)
print("Step 1: Train L1-Regularized Model")
print("="*60)

from sklearn.preprocessing import StandardScaler

# Scale features for L1 regularization (important for fair comparison)
scaler = StandardScaler()
X_v2_scaled = scaler.fit_transform(X_v2)
X_test_v2_scaled = scaler.transform(X_test_v2)

# Use L1 penalty (Lasso) to shrink unimportant coefficients to zero
clf_l1 = LogisticRegression(
    penalty='l1',
    C=1.0,  # Regularization strength (smaller = stronger regularization)
    solver='liblinear',  # Required for L1
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

clf_l1.fit(X_v2_scaled, y)

# Get feature coefficients
coefficients = clf_l1.coef_[0]
abs_coefficients = np.abs(coefficients)

# Create feature importance dataframe
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': abs_coefficients
}).sort_values('abs_coefficient', ascending=False)

print("\nFeature Importance (sorted by absolute coefficient):")
print(feature_importance_df.to_string(index=False))

# Visualize feature importance
print(f"\n{'='*60}")
print("Feature Importance Visualization")
print(f"{'='*60}")
top_n = 15
print(f"\nTop {top_n} Most Important Features:")
for idx, row in feature_importance_df.head(top_n).iterrows():
    bar_length = int(abs(row['abs_coefficient']) * 50)
    bar = '█' * bar_length
    print(f"{row['feature']:25s} | {bar} {row['abs_coefficient']:.4f}")

In [ ]:
# ============================================================================
# STEP 2: REMOVE NOISY/UNIMPORTANT FEATURES
# ============================================================================

print("\n" + "="*80)
print("Step 2: Feature Selection - Remove Noisy Features")
print("="*80)

# Strategy 1: Remove features with near-zero coefficients
threshold = 0.05  # Remove features with abs coefficient < threshold

important_features_mask = abs_coefficients >= threshold
n_features_removed = np.sum(~important_features_mask)
n_features_kept = np.sum(important_features_mask)

print(f"\nThreshold for feature selection: {threshold}")
print(f"Features removed: {n_features_removed}")
print(f"Features kept: {n_features_kept}")
print(f"Reduction: {100 * n_features_removed / len(feature_names):.1f}%")

if n_features_removed > 0:
    print(f"\nRemoved features:")
    removed_features = feature_importance_df[feature_importance_df['abs_coefficient'] < threshold]
    for idx, row in removed_features.iterrows():
        print(f"  - {row['feature']:25s} (coef: {row['coefficient']:7.4f})")
    
    print(f"\nKept features:")
    kept_features = feature_importance_df[feature_importance_df['abs_coefficient'] >= threshold]
    for idx, row in kept_features.iterrows():
        print(f"  - {row['feature']:25s} (coef: {row['coefficient']:7.4f})")

# Create reduced feature sets
X_v2_reduced = X_v2_scaled[:, important_features_mask]
X_test_v2_reduced = X_test_v2_scaled[:, important_features_mask]

print(f"\nOriginal feature shape: {X_v2_scaled.shape}")
print(f"Reduced feature shape: {X_v2_reduced.shape}")

# Strategy 2: Also try selecting top K features
print("\n" + "="*60)
print("Alternative: Select Top K Features")
print("="*60)

for top_k in [10, 15, 20, 25]:
    if top_k <= len(feature_names):
        print(f"\nTop {top_k} features would be:")
        for idx, row in feature_importance_df.head(top_k).iterrows():
            print(f"  {row['feature']}")


In [ ]:
# ============================================================================
# STEP 3: RETRAIN MODELS WITH REDUCED FEATURE SET
# ============================================================================

print("\n" + "="*80)
print("Step 3: Retrain Models with Reduced Feature Set")
print("="*80)

# Train Logistic Regression with reduced features
print("\n" + "="*60)
print("Logistic Regression (Reduced Features)")
print("="*60)

clf_reduced = LogisticRegressionCV(
    Cs=10,
    cv=5,
    scoring='roc_auc',
    max_iter=1000,
    n_jobs=-1,
    class_weight='balanced',
    random_state=42
)

clf_reduced.fit(X_v2_reduced, y)

test_probs_reduced = clf_reduced.predict_proba(X_test_v2_reduced)[:,1]
test_preds_reduced = clf_reduced.predict(X_test_v2_reduced)

print(f"Best C: {clf_reduced.C_[0]:.4f}")
print(f"CV AUC: {clf_reduced.scores_[1].mean(axis=0).max():.4f}")

print(f"\nTest Performance (Reduced Features):")
print(classification_report(y_test, test_preds_reduced))
test_auc_reduced = roc_auc_score(y_test, test_probs_reduced)
print(f"Test AUC: {test_auc_reduced:.4f}")

# Train XGBoost with reduced features
if test_probs_xgb is not None:
    print("\n" + "="*60)
    print("XGBoost (Reduced Features)")
    print("="*60)
    
    xgb_model_reduced = xgb.XGBClassifier(
        n_estimators=200,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1.0,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        eval_metric='auc'
    )
    
    xgb_model_reduced.fit(X_v2_reduced, y)
    test_probs_xgb_reduced = xgb_model_reduced.predict_proba(X_test_v2_reduced)[:,1]
    test_auc_xgb_reduced = roc_auc_score(y_test, test_probs_xgb_reduced)
    
    print(f"Test AUC: {test_auc_xgb_reduced:.4f}")
else:
    test_auc_xgb_reduced = None

# Train ensemble with reduced features
print("\n" + "="*60)
print("Ensemble (Reduced Features)")
print("="*60)

ensemble_reduced_models = [
    ('logistic', LogisticRegression(C=clf_reduced.C_[0], max_iter=1000, class_weight='balanced', random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=8, min_samples_leaf=10,
                                   class_weight='balanced', random_state=42, n_jobs=-1)),
    ('gb', GradientBoostingClassifier(n_estimators=150, max_depth=4, learning_rate=0.05,
                                       subsample=0.8, random_state=42))
]

if test_probs_xgb is not None:
    ensemble_reduced_models.append(('xgb', xgb_model_reduced))

ensemble_reduced = VotingClassifier(
    estimators=ensemble_reduced_models,
    voting='soft'
)

ensemble_reduced.fit(X_v2_reduced, y)
test_probs_ensemble_reduced = ensemble_reduced.predict_proba(X_test_v2_reduced)[:,1]
test_auc_ensemble_reduced = roc_auc_score(y_test, test_probs_ensemble_reduced)

print(f"Test AUC: {test_auc_ensemble_reduced:.4f}")


In [ ]:
# ============================================================================
# STEP 4: COMPARE FULL vs REDUCED FEATURE SETS
# ============================================================================

print("\n" + "="*80)
print("FEATURE SELECTION RESULTS COMPARISON")
print("="*80)

comparison_data = []

# Logistic Regression
comparison_data.append({
    'Model': 'Logistic Regression',
    'Full Features': test_auc_full,
    'Reduced Features': test_auc_reduced,
    'Difference': test_auc_reduced - test_auc_full,
    'Improvement %': 100 * (test_auc_reduced - test_auc_full) / test_auc_full
})

# XGBoost
if test_auc_xgb is not None and test_auc_xgb_reduced is not None:
    comparison_data.append({
        'Model': 'XGBoost',
        'Full Features': test_auc_xgb,
        'Reduced Features': test_auc_xgb_reduced,
        'Difference': test_auc_xgb_reduced - test_auc_xgb,
        'Improvement %': 100 * (test_auc_xgb_reduced - test_auc_xgb) / test_auc_xgb
    })

# Ensemble
comparison_data.append({
    'Model': 'Ensemble',
    'Full Features': test_auc_ensemble,
    'Reduced Features': test_auc_ensemble_reduced,
    'Difference': test_auc_ensemble_reduced - test_auc_ensemble,
    'Improvement %': 100 * (test_auc_ensemble_reduced - test_auc_ensemble) / test_auc_ensemble
})

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("Test AUC Comparison")
print("="*80)
print(comparison_df.to_string(index=False))

print("\n" + "="*80)
print("Summary")
print("="*80)
print(f"\nNumber of features:")
print(f"  Full feature set: {X_v2.shape[1]}")
print(f"  Reduced feature set: {X_v2_reduced.shape[1]}")
print(f"  Reduction: {100 * (X_v2.shape[1] - X_v2_reduced.shape[1]) / X_v2.shape[1]:.1f}%")

# Determine if feature selection helped
best_full_auc = max(test_auc_full, test_auc_ensemble, test_auc_xgb if test_auc_xgb is not None else 0)
best_reduced_auc = max(test_auc_reduced, test_auc_ensemble_reduced, test_auc_xgb_reduced if test_auc_xgb_reduced is not None else 0)

print(f"\nBest performance:")
print(f"  With all features: {best_full_auc:.4f}")
print(f"  With reduced features: {best_reduced_auc:.4f}")

if best_reduced_auc > best_full_auc:
    print(f"\n✅ Feature selection IMPROVED performance by {100*(best_reduced_auc - best_full_auc)/best_full_auc:.2f}%")
    print("   Removing noisy features helped generalization!")
elif abs(best_reduced_auc - best_full_auc) < 0.001:
    print(f"\n➡️  Feature selection maintained performance ({abs(best_reduced_auc - best_full_auc):.4f} difference)")
    print(f"   Achieved similar results with {100 * (X_v2.shape[1] - X_v2_reduced.shape[1]) / X_v2.shape[1]:.0f}% fewer features!")
else:
    print(f"\n⚠️  Feature selection DECREASED performance by {100*(best_full_auc - best_reduced_auc)/best_full_auc:.2f}%")
    print("   All features contribute to generalization in this case.")

# Save best submission from feature selection experiment
if best_reduced_auc >= best_full_auc:
    # Use reduced features
    best_reduced_probs = test_probs_ensemble_reduced if test_auc_ensemble_reduced == best_reduced_auc else test_probs_reduced
    submission_fs = test[["row_id"]].copy()
    submission_fs["prediction"] = best_reduced_probs
    submission_fs.to_csv("submission_feature_selected.csv", index=False)
    print(f"\n💾 Saved submission_feature_selected.csv with reduced features (AUC: {best_reduced_auc:.4f})")


In [ ]:
# ============================================================================
# BONUS: REGULARIZATION STRENGTH ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("EXPLORING DIFFERENT REGULARIZATION STRENGTHS")
print("="*80)

# Try different C values (inverse of regularization strength)
# Smaller C = stronger regularization = more features set to 0
C_values = [0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0]

regularization_results = []

for C_val in C_values:
    clf_temp = LogisticRegression(
        penalty='l1',
        C=C_val,
        solver='liblinear',
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    )
    
    clf_temp.fit(X_v2_scaled, y)
    
    # Count non-zero coefficients
    coeffs = clf_temp.coef_[0]
    n_nonzero = np.sum(np.abs(coeffs) > 0.01)  # Count features with |coef| > 0.01
    n_zero = len(coeffs) - n_nonzero
    
    # Evaluate performance
    test_probs_temp = clf_temp.predict_proba(X_test_v2_scaled)[:,1]
    test_auc_temp = roc_auc_score(y_test, test_probs_temp)
    
    regularization_results.append({
        'C': C_val,
        'Regularization Strength (1/C)': 1/C_val,
        'Non-zero Features': n_nonzero,
        'Zero Features': n_zero,
        'Test AUC': test_auc_temp
    })

reg_df = pd.DataFrame(regularization_results)

print("\nRegularization Strength vs. Feature Count vs. Performance:")
print(reg_df.to_string(index=False))

print("\n" + "="*60)
print("Key Insights:")
print("="*60)

best_auc_idx = reg_df['Test AUC'].idxmax()
best_row = reg_df.iloc[best_auc_idx]

print(f"\nBest performance:")
print(f"  C = {best_row['C']}")
print(f"  Features used: {best_row['Non-zero Features']}/{len(feature_names)}")
print(f"  Test AUC: {best_row['Test AUC']:.4f}")

# Find the most parsimonious model (fewest features with good performance)
# Define "good performance" as within 0.5% of best AUC
auc_threshold = best_row['Test AUC'] * 0.995
parsimonious_candidates = reg_df[reg_df['Test AUC'] >= auc_threshold]
if len(parsimonious_candidates) > 0:
    parsimonious_idx = parsimonious_candidates['Non-zero Features'].idxmin()
    parsimonious_row = reg_df.iloc[parsimonious_idx]
    
    print(f"\nMost parsimonious model (within 0.5% of best AUC):")
    print(f"  C = {parsimonious_row['C']}")
    print(f"  Features used: {parsimonious_row['Non-zero Features']}/{len(feature_names)}")
    print(f"  Test AUC: {parsimonious_row['Test AUC']:.4f}")
    print(f"  Reduction: {100 * (len(feature_names) - parsimonious_row['Non-zero Features']) / len(feature_names):.1f}%")

print("\n" + "="*60)
print("Recommendations:")
print("="*60)
print("\n1. Balance between complexity and performance:")
print("   - Too few features: Underfitting, poor generalization")
print("   - Too many features: Overfitting, noisy predictions")
print(f"\n2. For this dataset, using ~{best_row['Non-zero Features']:.0f}-{parsimonious_row['Non-zero Features']:.0f} features")
print("   achieves optimal trade-off between model complexity and performance.")
print("\n3. The most important features identified by L1 regularization are:")
print("   those with consistently high coefficients across different C values.")


In [ ]:
# ============================================================================
# IDENTIFY THE 14 SELECTED FEATURES AT OPTIMAL C=0.05
# ============================================================================

print("\n" + "="*80)
print("OPTIMAL FEATURE SET ANALYSIS (C=0.05)")
print("="*80)

# Train model with optimal C
optimal_C = 0.05
clf_optimal = LogisticRegression(
    penalty='l1',
    C=optimal_C,
    solver='liblinear',
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

clf_optimal.fit(X_v2_scaled, y)

# Get coefficients
optimal_coeffs = clf_optimal.coef_[0]
optimal_abs_coeffs = np.abs(optimal_coeffs)

# Create detailed feature analysis
optimal_feature_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': optimal_coeffs,
    'abs_coefficient': optimal_abs_coeffs,
    'selected': optimal_abs_coeffs > 0.01  # Threshold for "selected"
}).sort_values('abs_coefficient', ascending=False)

# Separate selected and removed features
selected_features = optimal_feature_df[optimal_feature_df['selected']]
removed_features = optimal_feature_df[~optimal_feature_df['selected']]

print(f"\n{'='*80}")
print(f"SELECTED FEATURES ({len(selected_features)} features)")
print(f"{'='*80}")
print("\nThese features CONTRIBUTE to detecting rule violations:\n")

for idx, row in selected_features.iterrows():
    sign = "+" if row['coefficient'] > 0 else "-"
    bar_length = int(row['abs_coefficient'] * 50)
    bar = '█' * bar_length
    
    # Add interpretation
    if row['coefficient'] > 0:
        effect = "→ INCREASES violation probability"
    else:
        effect = "→ DECREASES violation probability"
    
    print(f"{row['feature']:25s} | {sign} | {bar} {row['abs_coefficient']:.4f} | {effect}")

print(f"\n{'='*80}")
print(f"REMOVED FEATURES ({len(removed_features)} features)")
print(f"{'='*80}")
print("\nThese features were NOISY and hurt generalization:\n")

for idx, row in removed_features.iterrows():
    print(f"  - {row['feature']:25s} (coef: {row['coefficient']:7.4f})")

# Performance with optimal features
X_v2_optimal = X_v2_scaled[:, optimal_feature_df['selected'].values]
X_test_v2_optimal = X_test_v2_scaled[:, optimal_feature_df['selected'].values]

test_probs_optimal = clf_optimal.predict_proba(X_test_v2_scaled)[:,1]
test_auc_optimal = roc_auc_score(y_test, test_probs_optimal)

print(f"\n{'='*80}")
print("PERFORMANCE")
print(f"{'='*80}")
print(f"\nTest AUC with optimal features: {test_auc_optimal:.4f}")
print(f"Previous best (Ensemble, 35 features): {test_auc_ensemble:.4f}")
print(f"Improvement: {test_auc_optimal - test_auc_ensemble:.4f} ({100*(test_auc_optimal - test_auc_ensemble)/test_auc_ensemble:.2f}%)")

# Save optimal submission
submission_optimal = test[["row_id"]].copy()
submission_optimal["prediction"] = test_probs_optimal
submission_optimal.to_csv("submission_optimal_l1.csv", index=False)
print(f"\n✅ Saved submission_optimal_l1.csv with 14 selected features!")

# Print feature categories breakdown
print(f"\n{'='*80}")
print("FEATURE CATEGORY BREAKDOWN")
print(f"{'='*80}")

categories = {
    'Similarity Features': ['sim_pos1', 'sim_pos2', 'sim_neg1', 'sim_neg2', 'sim_rule'],
    'Aggregate Features': ['pos_avg', 'neg_avg', 'pos_max', 'neg_max', 'pos_min', 'neg_min'],
    'Ratio/Difference': ['pos_neg_ratio', 'pos_neg_diff', 'pos_range', 'neg_range'],
    'Rule-Example Similarity': ['rule_pos_avg_sim', 'rule_neg_avg_sim'],
    'Cross-Similarity': ['pos1_pos2_sim', 'neg1_neg2_sim'],
    'Distance Features': ['dist_to_pos', 'dist_to_neg', 'dist_ratio'],
    'Text Statistics': ['num_chars', 'num_words', 'has_url', 'caps_ratio'],
    'Categorical': ['subreddit_encoded']
}

selected_names = set(selected_features['feature'].values)

print("\nWhich feature types were most valuable?\n")
for category, feats in categories.items():
    selected_count = sum(1 for f in feats if f in selected_names)
    total_count = len(feats)
    pct = 100 * selected_count / total_count if total_count > 0 else 0
    
    status = "✓" if selected_count > 0 else "✗"
    print(f"{status} {category:25s}: {selected_count}/{total_count} selected ({pct:.0f}%)")
    
    # Show which specific features from this category were selected
    if selected_count > 0:
        selected_from_cat = [f for f in feats if f in selected_names]
        for feat in selected_from_cat:
            coef = selected_features[selected_features['feature'] == feat]['coefficient'].values[0]
            print(f"    → {feat:25s} (coef: {coef:+.4f})")
